In [ ]:
!cp /content/drive/MyDrive/data/learn_cd4.fa /content
!cp /content/drive/MyDrive/data/learn_cd4_act.txt /content

In [1]:
!cp /content/drive/MyDrive/data/learn_H7-hESC.fa /content
!cp /content/drive/MyDrive/data/learn_H7-hESC_act.txt /content

In [ ]:
# The function take input from the pickle files generated in previous stage for seqs and targets.
# #6 There is some kind of problem in the HDF5 that the strings are not stored to store strings
# or array/list of strings we have to mention that it indeed a string 'S'. See cells further down.


In [2]:
import numpy.random as npr
import numpy as np
import sys
import h5py

def seq_hdf5(fasta_file,target_files,out_file,seqs,targets):

  counts=False
  test_pct=0.05
  valid_pct=0.05
  add_feature_file=False
  valid_test=False
  batch_size=None

  seqs=seqs.reshape((seqs.shape[0],4,1,int(seqs.shape[1]/4))) #1

  headers=[]
  for line in open(fasta_file):
    if line[0]=='>':
      headers.append(line[1:].rstrip())
  headers=np.array(headers,dtype='S') #6

  target_labels=open(target_files).readline().strip().split('\t') #2
  target_labels=np.array(target_labels,dtype='S') #6

  if add_feature_file:
    df_add=pd.read_table(add_features_file,index_col=0) #df: dataframe
    df_add=df_add.astype(np.float32,copy=False)

  order=npr.permutation(seqs.shape[0])
  seqs=seqs[order]
  targets=targets[order]
  headers=headers[order]
  if add_feature_file:
    df_add=df_add.iloc[order]

  if counts:
    assert(test_pct+valid_pct <= seqs.shape[0])
  else:
    assert(test_pct+valid_pct <=1.0)

  if counts: #3
    test_count=int(test_pct)
    valid_count=int(valid_count)
  else:
    test_count=int(0.5+test_pct*seqs.shape[0])
    valid_count=int(0.5+valid_pct*seqs.shape[0])

  train_count=seqs.shape[0]- (test_count+valid_count)
  train_count=batch_round(train_count,batch_size) #5
  print('%d training sequences'% train_count,file=sys.stderr)

  valid_count=batch_round(valid_count,batch_size)
  print('%d validation sequences'% valid_count,file=sys.stderr)

  test_count=batch_round(test_count,batch_size)
  print('%d test sequences'% test_count,file=sys.stderr)
  
  i=0
  train_seqs,train_targets=seqs[i:i+train_count,:],targets[i:i+train_count,:]
  i+=train_count
  valid_seqs,valid_targets,valid_headers=seqs[i:i+valid_count,:],targets[i:i+valid_count,:],headers[i:i+valid_count]
  i+=valid_count
  test_seqs,test_targets,test_headers=seqs[i:i+test_count,:],targets[i:i+test_count,:],headers[i:i+test_count]

  if add_feature_file:
    i=0
    train_add=df_add.iloc[i:i+train_count]
    i+=train_count
    valid_add=df.iloc[i:i+valid_count]
    i+=valid_count
    test_add=df.iloc[i:i+test_count]

  h5f=h5py.File(out_file,'w')
  h5f.create_dataset('target_labels',data=target_labels)

  if train_count > 0:
    h5f.create_dataset('train_in',data=train_seqs)
    h5f.create_dataset('train_out',data=train_targets)
  if valid_count > 0:
    h5f.create_dataset('valid_in',data=valid_seqs)
    h5f.create_dataset('valid_out',data=valid_targets)
  if test_count > 0:
    h5f.create_dataset('test_in',data=test_seqs)
    h5f.create_dataset('test_out',data=test_targets)
    h5f.create_dataset('test_headers',data=test_headers)
  elif valid_test: #4
    hf5.create_dataset('test_in',data=valid_seqs)
    hf5.create_dataset('test_out',data=valid_target)
    h5f.create_dataset('test_headers',data=valid_hesders)

  if add_feature_file:
    h5f.create_dataset('add_labels',data=list(df_add.columns))

    if train_count > 0:
      h5f.create_dataset('train_add',data=train_add.as_matrix())
    if valid_count > 0:
      h5f.create_dataset('valid_add',data=valid_add.as_matrix())
    if test_count > 0:
      h5f.create_dataset('test_add',data=test_add.as_matrix())
    elif valid_test:
      h5f.create_dataset('test_add',data=valid_add.as_matrix())

  h5f.close()



In [3]:
def batch_round(count,batch_size):
  if batch_size!=None:
    count-=(batch_size%count)
  return count

In [4]:
import pickle
seqs=pickle.load(open('/content/drive/MyDrive/data/seqs_H7-hESC.pkl','rb'))
targets=pickle.load(open('/content/drive/MyDrive/data/targets_H7-hESC.pkl','rb'))

In [5]:
seqs.shape, targets.shape, type(targets)

((517655, 2400), (517655, 1), numpy.ndarray)

In [ ]:
seqs.shape, targets.shape, type(targets)

((47605, 2400), (47605, 165), numpy.ndarray)

In [ ]:
seqs.shape, targets.shape, type(targets)

((91544, 2400), (91544, 1), numpy.ndarray)

In [ ]:
seq_hdf5('/content/learn_cd4.fa','/content/learn_cd4_act.txt','learn_cd4.h5',seqs,targets)

In [6]:
seq_hdf5('/content/learn_H7-hESC.fa','/content/learn_H7-hESC_act.txt','learn_H7-hESC.h5',seqs,targets)

465889 training sequences
25883 validation sequences
25883 test sequences


In [7]:
!cp /content/learn_H7-hESC.h5 /content/drive/MyDrive/data/

In [ ]:
h5f=h5py.File('/content/learn_cd4.h5','r')

In [ ]:
a=h5f['target_labels'][0]
h5f['train_in'][1]

array([[[False, False, False, ..., False, False,  True]],

       [[ True, False, False, ..., False,  True, False]],

       [[False, False,  True, ...,  True, False, False]],

       [[ True, False, False, ..., False, False,  True]]])

In [ ]:
# So HDF5 stores byte stream but to work with it is usually prefer to be in string
# format, it can be list/array. So to convert the byte to string the following code can be used.
# Also, keep in mind we can declare hdf5 attribute, it can be inserted in the above cell-3 as:
# h5f.attrs['test_headers'] = np.encode(np.array(list_of_strings, type='U'), encoding='utf8').
l=[str(s,'utf=8') for s in h5f['test_headers']]

In [ ]:
type(l)

list

In [ ]:
l=l=[str(s,'utf=8') for s in h5f['target_labels']]
l

In [ ]:
h5f['test_out'][0]

array([1], dtype=int8)

In [ ]:
h5f['train_out'][37004]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int8)

In [ ]:
h5f.close()